# Accessing Level-1 and ARD (A)SAR Data from ERS-1/2 and ENVISAT

This notebook serves as a guide for accessing and downloading the ERS/Envisat (A)SAR Level-1 and ARD datasets. These datasets are made available through the EODC STAC API and Data Access Service, enabling open and free access to end users. The primary goal is to increase the visibility and usability of heritage Earth observation data from the ERS and Envisat missions. This guide will walk you through the steps for discovering, accessing, and downloading the data.

In accordance with ESA's data sharing policy and **terms and conditions**, users must accept the ESA data license to access and download the data.

## Prerequisites

Before we dive into the data discovery process, let's ensure we have all the necessary libraries installed and imported. We'll be using `pystac-client` for interacting with the STAC API and `requests` for handling HTTP requests.

### Make Sure the Following Libraries are Installed

Ensure that you have the following libraries installed in your Python environment:

- `pystac-client`
- `requests`

### Import Libraries



In [1]:
import requests
from pystac_client import Client

## Step 1: Data Discovery

In this section, we will explore how to discover the ERS/Envisat (A)SAR Level-1 and ARD datasets using the EODC STAC API. The EODC STAC API allows users to search for specific data collections and items, view metadata, and obtain URLs for individual assets.

To get started, we use the `Client` class to connect to the EODC STAC API endpoint, i.e., https://stac.eodc.eu/api/v1.

In [ ]:
# Define the API URL
api_url = "https://stac.eodc.eu/api/v1"

# Define the STAC collection ID 
collection_id = "ASA_IMP_1P" # Other available collection IDs: "SAR_IMP_1P", "SAR_IMS_1P", "ASA_IMS_1P", "ERS_ENVISAT_NRB"

# Define the area of interest (AOI) as a bounding box
bbox = [9.25, 46.31, 17.46, 49.18]

# Define the time range for the search
time_range = "2002-01-01/2002-10-31"

# Open the STAC catalog using the specified API URL
eodc_catalog = Client.open(api_url)

# Perform a search in the catalog with the specified parameters
search = eodc_catalog.search(
    max_items=5,                # Maximum number of items to return
    collections=collection_id,  # The collection to search within
    bbox=bbox,                  # The area of interest
    datetime=time_range         # The time range for the search
)

# Collect the found items into an item collection
items = search.item_collection()

print(f"\nItems in Collection '{collection_id}':")
for item in items:
    print(f"- {item.id}")

# Obtain Asset URLs for a specific item
item_id = "ASA_IMP_1PREOD20021031_205825_000000342010_00444_03504_0000"
item = eodc_catalog.get_collection(collection_id).get_item(item_id)
print(f"\nAssets for Item '{item_id}':")
for asset_key, asset in item.assets.items():
    print(f"- {asset_key}: {asset.href}")


Items in Collection 'ASA_IMP_1P':
- ASA_IMP_1PREOD20021031_205825_000000342010_00444_03504_0000
- ASA_IMP_1PREOD20021022_092004_000000472010_00308_03368_0000
- ASA_IMP_1PREOD20021018_094556_000000542010_00251_03311_0000

Assets for Item 'ASA_IMP_1PREOD20021031_205825_000000342010_00444_03504_0000':
- sip: https://data.eodc.eu/collections/ASA_IMP_1P/imp/EN1_REOD_ASA_IMP_1P_20021031T205825_20021031T205900_003504_0444_0981_20230513T142317_v0101.ZIP/ASA_IMP_1PREOD20021031_205825_000000342010_00444_03504_0000.SI.XML
- eosip-zip: https://data.eodc.eu/collections/ASA_IMP_1P/imp/EN1_REOD_ASA_IMP_1P_20021031T205825_20021031T205900_003504_0444_0981_20230513T142317_v0101.ZIP
- thumbnail: https://data.eodc.eu/collections/ASA_IMP_1P/imp/EN1_REOD_ASA_IMP_1P_20021031T205825_20021031T205900_003504_0444_0981_20230513T142317_v0101.ZIP/ASA_IMP_1PREOD20021031_205825_000000342010_00444_03504_0000.BI.JPG
- eo-product: https://data.eodc.eu/collections/ASA_IMP_1P/imp/EN1_REOD_ASA_IMP_1P_20021031T205825_20021

## Step 2: Data Access Service

In this step, we will generate a signed URL using the Data Access Service. This ensures compliance with ESA's data sharing policy and terms and conditions.

### Generate Signed URL

To generate a signed URL, we need to send a POST request to the Data Access Service endpoint with the required query parameters. We'll use the `requests` library for this purpose.

**Note:** The Data Access Service URL accepts ESA's terms and conditions here, hence the last endpoint `/ESA`.

In [3]:
# Define the Data Access Service URL and the required parameters
data_access_service_url = "https://access.dev.services.eodc.eu/termsandconditions/ESA"
hda_url = item.assets["eo-product"].href
esa_license_url = "https://earth.esa.int/eogateway/terms-and-conditions"
payload = {
    "hda_url": hda_url,
    "esa_license": esa_license_url,
    "accept_esa_license": "yes"
}

# Send a POST request to generate the signed URL
response = requests.post(data_access_service_url, params=payload)
signed_url = response.json()["signed_urls"][0]
print(f"\nSigned URL: {signed_url}")


Signed URL: https://data.eodc.eu/collections/ASA_IMP_1P/imp/EN1_REOD_ASA_IMP_1P_20021031T205825_20021031T205900_003504_0444_0981_20230513T142317_v0101.ZIP/ASA_IMP_1PREOD20021031_205825_000000342010_00444_03504_0000.N1?x-hda-expire=20250224T145135&x-hda-sid=67aac23af2f7b6d9&x-hda-uid=NzBMTmQ2MTZaMFlhaXZ1eXRTSUx1YmhtTUNoVFlEVHI=&x-hda-signature=b3e651a43be609e34e465312cc006b4c26e02ac46bc7a00c974adc783349b954


## Step 3: Data Downloading

In this final step, we will use the signed URL to download the data. The signed URL is valid for 3 days from its generation, as indicated by the "x-hda-expire" parameter.

### Download Data

To download the data, we will use the `requests` library to send a GET request to the signed URL and save the response content to a file.

In [6]:
# Define the output filename
output_filename = "/tmp/ASA_IMP_1PREOD20021031_205825_000000342010_00444_03504_0000.N1"

# Send a GET request to the signed URL and save the response content to a file
response = requests.get(signed_url)
with open(output_filename, 'wb') as f:
    f.write(response.content)

print(f"\nData downloaded and saved as '{output_filename}'")


Data downloaded and saved as '/tmp/ASA_IMP_1PREOD20021031_205825_000000342010_00444_03504_0000.N1'


### Handling Expired URLs

If the signed URL expires, you can generate a new one by repeating Step 2. Simply send another POST request to the Data Access Service endpoint to obtain a new signed URL.